In [28]:
import pandas as pd
import json
import numpy as np
from os import listdir
from io import StringIO
from Bio import Phylo, AlignIO
from Bio.Alphabet import generic_dna
import re

path="/home/maeperez/scratch/Clams/hyphy2.5/gard_LCB/"
aln_breakdown='../mauve_alignments/core_ALL_alignments/'
path_out="/home/maeperez/projects/def-bacc/maeperez/Clams/VesicSymb_Evolution/dowstream_analyses_and_data/"


In [29]:

genes=[]
for file in listdir(path):
    if file[-4:]=="json":
        genes+=[str(file.split('.')[0])]
genes=sorted(genes)
print(len(genes))


715


In [27]:
path_to_files='../mauve_alignments/core_ALL_alignments/'
with open(path_to_files+'filelist','r') as f:
    lines=[file[8:-6] for file in f.read().splitlines()]

failed_gard=sorted(set(lines)-set(genes))
failed_gard_idx=[lines.index(gene)+1 for gene in failed_gard]
print(len(failed_gard))
print('\n'.join([' '.join(map(str,item)) for item in list(zip(failed_gard_idx,failed_gard))]))
','.join(map(str,failed_gard_idx))


1
592 LCB_592


'592'

In [17]:
filenames=[]
with_recombination=[]
for gene in [genes[1],genes[555]]:
    remote_file = open(path+gene+'.GARD.json')
    gard_json = json.load(remote_file)
    breakpoints=[0]+[gard_json['input']['number of sites']]
    
    if 'improvements' not in gard_json.keys():
        continue
        
    else: 
        best_AICc=sum([gard_json['improvements'][bp]['deltaAICc'] for bp in gard_json['improvements'].keys()])+gard_json['baselineScore']
        AICc_ratio=float(gard_json['singleTreeAICc'])/best_AICc
        if AICc_ratio>=1:
            with_recombination+=[gene]
            
            ### Find all puttive breakpoints
            breakpoints=[0]+[gard_json['breakpointData'][k]['bps'][0][1] for k in gard_json['breakpointData'].keys()]
            print(breakpoints)
            
    ### break alignment at all putative breakpoints ###
    alignment = AlignIO.read(path_to_files+gene+'.fasta', "fasta",alphabet=generic_dna)
    start=breakpoints.pop(0)
    sub=0
    while len(breakpoints)>0:
        sub+=1
        end=breakpoints.pop(0)
        new_aln= alignment[:,start:end]

        for record in new_aln:
            record.id = record.id.split('/')[-2].split('_')[0].replace('.gbk','')

        new_aln.sort()
        filename=gene+'_'+str(sub)+'_'+str(start)+'to'+str(end)+'.nex'
        filenames+=filename

        AlignIO.write([new_aln],aln_breakdown+'nex/'+filename, "nexus")
        print(filename)

        start=end

print( 'number of genes that failed AICc test ',len(with_recombination),"/",len(genes))        

[0, 68, 1154]
LCB_002_1_0to68.nex
LCB_002_2_68to1154.nex
number of genes that failed AICc test  1 / 711


In [4]:
# with open(path_out+'core_LCB_with_recomb_AICc.txt','w') as f:
#     f.write('\n'.join(with_recombination))